In [1]:
using NPZ

include("WGR.jl")
using .WGR

import Base.flush

buf = WGR.BlockBuffer(128)
setBlocks(x,y,z,b) = WGR.setBlocks!(buf, x, y, z, b)
flush() = WGR.flush!(buf)

flush (generic function with 12 methods)

In [2]:
monu10 = npzread("monu10.npy")
shape = size(monu10)
println(shape)

(72, 122, 72)


In [3]:
nonz = findall(x -> x>0, monu10)
println(size(nonz))

(150764,)


In [4]:
function refreshNd(arr, pos, full = false)
    
    # Account for 1-based indexing
    pos = [p - 1 for p in pos]
    
    if full
        shape = size(arr)
        for x in 1:shape[1], y in 1:shape[2], z in 1:shape[3]
            setBlocks(x + pos[1], y + pos[2], z + pos[3], arr[x, y, z])
        end
    else
        nonz = findall(x -> x>0, arr)
        for p in nonz
            setBlocks(p[1] + pos[1], p[2] + pos[2], p[3] + pos[3], arr[p])
        end
    end
    
    flush()
end

refreshNd (generic function with 2 methods)

In [5]:
refreshNd(monu10, [1, 65, 1], true)

0

In [6]:
# Parameters
N = 3       # Pattern size
step = 1    # Pattern stride
mcsize = 28 # MCMC size
eps = 3e-4  # Epsilon
targetPos = [100, 65, 1] # Where to put the mcmc
nIters = 3000000

bDistλ = mcsize^3 * 25

548800

In [7]:
#Block Index
function blockIndex(blocks)
    blockDict = Dict{UInt32, Array{Int64}}(0 => [1, 0])
    blockRep = [0]
    for b in monu10
        if !haskey(blockDict, b)
            blockDict[b] = [length(blockDict) + 1, 0]
            push!(blockRep, b)
        end
        blockDict[b][2] += 1
    end
    
    return blockDict, blockRep
end

# Slice weights
mutable struct SliceWeights
    slices::Dict{Array{WGR.Block}, Int64}
    totalNum::Int64
end

In [8]:
SliceWeights() = SliceWeights(Dict{Array{WGR.Block}, Int64}(), 0)

import Base.getindex, Base.push!
function push!(sW::SliceWeights, pattern)
    if !haskey(sW.slices, pattern)
        sW.slices[pattern] = 0
    end
    sW.slices[pattern] += 1
    sW.totalNum += 1
end

Base.getindex(sW::SliceWeights, key) = haskey(sW.slices, key) ? (sW.slices[key] / sW.totalNum) : eps

In [9]:
function randBlock(choice, dist)
    return choice[rand(dist)]
end

randBlock (generic function with 1 method)

In [10]:
using Distributions

Nb = length(bDict)
bDict, bRep = blockIndex(monu10)

bTotal = sum([b[2] for b in values(bDict)])
choices = [bRep[b] for b in 1:Nb]
bDist = Categorical([bDict[bRep[b]][2] / bTotal for b in 1:Nb])

randBlock() = randBlock(choices, bDist)

LoadError: UndefVarError: bDict not defined

In [11]:
# Slice
function readSlices(arr, sW; N = 3, stride = 1)
    arrShape = size(arr)
    for x in 1:arrShape[1]-N, y in 1:arrShape[2]-N, z in 1:arrShape[3]-N
        nSlice = arr[x:x+N-1, y:y+N-1, z:z+N-1]
        # @show size(nSlice)
        for dx in [1:1:N, N:-1:1], dz in [1:1:N, N:-1:1]
            push!(sW, nSlice[dx, :, dz])
        end
    end
end

readSlices (generic function with 1 method)

In [ ]:
# Slice the array
sW = SliceWeights()
@time readSlices(monu10, sW; N, stride = step)

  1.591166 seconds (5.47 M allocations: 705.996 MiB, 6.15% gc time, 21.84% compilation time)


LoadError: InterruptException:

In [13]:
@show length(sW.slices)
@show sW[zeros(3,3,3)]

length(sW.slices) = 10782
sW[zeros(3, 3, 3)] = 0.6748988190109062


0.6748988190109062

In [14]:
flush()
# mcmc = [randBlock() for x in 1:mcsize, y in 1:mcsize, z in 1:mcsize]
mcmc = [bRep[2] for x in 1:mcsize, y in 1:mcsize, z in 1:mcsize]
refreshNd(mcmc, targetPos, true)

LoadError: UndefVarError: bRep not defined

In [ ]:
using ExactOptimalTransport
using LinearAlgebra
using Tulip

# Energy
E = 0
blockCountMCMC = []
mcmcBlocksCount = size(mcmc)[1] * size(mcmc)[2] * size(mcmc)[3]

W1Cost = ones(length(bDict), length(bDict)) - LinearAlgebra.I
lpOpt = Tulip.Optimizer()
# distDiff(p::Array{AbstractFloat}, q::Array{AbstractFloat}) = ExactOptimalTransport.emd2(p, q, W1Cost, lpOpt)
distDiff(p, q) = norm(p - q)

function totalE(arr, sW)
    
    shape = size(arr)
    E = 0.0
    
    # Count all patterns
    for (x, y, z) in Iterators.product([1:shape[i] - N for i in 1:3]...)
        nSlice = arr[x:x+N-1, y:y+N-1, z:z+N-1]
        E -= log(sW[nSlice])
    end
    
    # Block marginals
    for k in 1:Nb
        push!(blockCountMCMC, sum(arr .== bRep[k]))
    end
    bDistMCMC = blockCountMCMC ./ mcmcBlocksCount
    
    @show bDist.p
    @show bDistMCMC
    @show dd = distDiff(bDist.p, bDistMCMC)
    E += dd * bDistλ
    
    return E, dd * bDistλ
end

@show E, E_bDist = totalE(mcmc, sW)

LoadError: UndefVarError: mcmc not defined

In [ ]:
@show bDict
@show bRep
@show bDist.p
@show blockCountMCMC

LoadError: UndefVarError: bDict not defined

In [17]:
mutable struct MCMCState2
    arr::Array{WGR.Block}
    blockCount::Array{Int64}
    
    E::Float64
    E_dist::Float64
    
    accepted::Int64
    totalIters::Int64
end
MCMCState = MCMCState2

# TODO: remove duplicated patterns
function setCalcdE!(state, posArr, blkArr)

    dE = 0
    arr = state.arr
    shape = size(arr)
    
    # Old energy
    for (p, b) in zip(posArr, blkArr)
        
        starts = [max(p[i] - N + 1, 1) for i in 1:3]
        ends = [min(p[i], shape[i] - N + 1) for i in 1:3]
        
        for (x, y, z) in Iterators.product([s:e for (s,e) in zip(starts, ends)]...)
            nSlice = arr[x:x+N-1, y:y+N-1, z:z+N-1]
            dE += log(sW[nSlice])
        end

    end
    
    # Set blocks
    newbCount = copy(state.blockCount)
    for (p, b) in zip(posArr, blkArr)
        newbCount[bDict[arr[p...]][1]] -= 1
        arr[p...] = b
        newbCount[bDict[b][1]] += 1
    end
    
    # New energy
    for (p, b) in zip(posArr, blkArr)
    
        starts = [max(p[i] - N + 1, 1) for i in 1:3]
        ends = [min(p[i], shape[i]- N + 1) for i in 1:3]
        
        for (x, y, z) in Iterators.product([s:e for (s,e) in zip(starts, ends)]...)
            nSlice = arr[x:x+N-1, y:y+N-1, z:z+N-1]
            dE -= log(sW[nSlice])
        end

    end
    
    # Block distribution
    newbDist = newbCount ./ mcmcBlocksCount
    # @show sum(newbCount)
    # @show newbDist
    dd = distDiff(bDist.p, newbDist)
    dEdist = (dd * bDistλ - state.E_dist)
    dE += dEdist
    
    return dE, newbCount, dEdist

end

function mcmcIter!(state::MCMCState)

    state.totalIters += 1

    shape = size(state.arr)
    pos = [rand(1:shape[i]) for i in 1:3]
    blk = randBlock()
    
    original = state.arr[pos...]

    # Do the proposal and calculate dE
    dE, newbCount, dEDist = setCalcdE!(state, [pos], [blk])
    
    if dE > 0
        alpha = exp(-dE)
        if rand(Uniform(0, 1)) > alpha # Rejected
            state.arr[pos...] = original
            return false, nothing, nothing, nothing
        end
    end
    
    state.accepted += 1
    state.E += dE
    state.E_dist += dEDist
    state.blockCount = newbCount
    setBlocks((pos + targetPos .- 1)..., blk)
    
    return true, dE, [pos], [blk]
end

InterruptException: InterruptException:

In [18]:
rejected = 0

blockCount = [b[2] for b in values(bDict)]
state = MCMCState(mcmc, blockCountMCMC, E, E_bDist, 0, 0)

for it in 1:nIters
    accepted, dE, ps, bs = mcmcIter!(state)
        
    if it % (div(nIters, 100)) == 1
        println("Iter $it | E = $(state.E) (E_dist = $(state.E_dist)) | $(state.blockCount)")
        flush()
        flush(stdout)
    end
end

flush()

LoadError: UndefVarError: bDict not defined